In [1]:
import pandas as pd
import re
df =pd.read_excel('regex_data_public.xlsx',dtype='string')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   string
 1   email       994 non-null    string
 2   area        934 non-null    string
 3   phone       994 non-null    string
 4   date        997 non-null    string
dtypes: string(5)
memory usage: 39.2 KB


,Unnamed: 0,email,area,phone,date
0,0,v96,<NA>,0-5791-437,2018-10-12 00:00:00
1,1,leslierodriguez@miller-davenport.net,E2,02477-13974,2019-07-05 00:00:00
2,2,hscott@duran.com,ตะวันตก,091617714,2021-01-11 00:00:00
3,3,owilson@cochran-montgomery.com,ภาคเหนือ,0-9798-57785,2019-02-12 00:00:00
4,4,ryan15@morrison.biz,ภาคกลาง,07440-312,mkc5K1v45
...,...,...,...,...,...
995,995,barnesstephen@hodges.com,ตะวันออก,0-2188104,2019-11-16 00:00:00
996,996,daniel60@taylor.com,ใต้,0-20247303,2019-10-31 00:00:00
997,997,uturner@jones.com,กลาง,07520-114,2reBGNs
998,998,sullivankimberly@perkins.com,ตะวันออก,02172-065,2019-03-24 00:00:00


In [2]:
# Q1 Validation functions for emails
def is_valid_email(email):
    if isinstance(email, str):
        pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        return re.match(pattern, email) is not None
    else:
        return False

df_email = df[['email']].drop_duplicates()
df_email['valid_email'] = df_email['email'].apply(is_valid_email)
df_email.info()
df_email.head()
df_email['valid_email'].sum()

<class 'pandas.core.frame.DataFrame'>
Index: 995 entries, 0 to 999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   email        994 non-null    string
 1   valid_email  995 non-null    bool  
dtypes: bool(1), string(1)
memory usage: 16.5 KB


907

In [3]:
# Define the dictionary for replacements
replacement_dict = {
    'กลาง': 'ภาคกลาง',
    'ภาคกลาง': 'ภาคกลาง',
    'ภาคกลาง ': 'ภาคกลาง',
    'เหนือ': 'ภาคเหนือ',
    'ภาคเหนือ': 'ภาคเหนือ',
    'ใต้': 'ภาคใต้',
    'ภาคใต้ ': 'ภาคใต้',
    'ตะวันออก': 'ภาคตะวันออก',
    'ภาคตะวันออก': 'ภาคตะวันออก',
    'ภาคตะวันออก ': 'ภาคตะวันออก',
    'ตะวันตก': 'ภาคตะวันตก',
    'ตะวันตก\r': 'ภาคตะวันตก',
    'ตะวันตก\n': 'ภาคตะวันตก',
    'ตะวันตก\r\n': 'ภาคตะวันตก',
    'ตะวันออกเฉียงเหนือ': 'ภาคตะวันออกเฉียงเหนือ',
    'ภาคตะวันออกเฉียงเหนือ': 'ภาคตะวันออกเฉียงเหนือ',
    '': ''
}

# Preprocessing function for 'area'
def preprocess_area(area):
    if isinstance(area, str):
        # Trim whitespace and newline characters
        area_cleaned = re.sub(r'^\s+|\s+$', '', area)
        # Replace using the dictionary
        return replacement_dict.get(area_cleaned, area_cleaned)
    else:
        return area


# Apply preprocessing to the 'area' column
df['area_preprocessed'] = df['area'].apply(preprocess_area)

# Display the first few rows of the dataframe with preprocessed area column
df[['area', 'area_preprocessed']].head()

,area,area_preprocessed
0,<NA>,<NA>
1,E2,E2
2,ตะวันตก,ภาคตะวันตก
3,ภาคเหนือ,ภาคเหนือ
4,ภาคกลาง,ภาคกลาง


In [4]:
# Q2 Validation functions for thai area
d = ['ภาคกลาง',
     'ภาคเหนือ',
     'ภาคใต้',
     'ภาคตะวันออก',
     'ภาคตะวันตก',
     'ภาคตะวันออกเฉียงเหนือ'
    ]

# Regular expression pattern for the area column
# We use the keys of the dictionary, escape any special regex characters, and allow for optional whitespace/newlines
regex_patterns = '|'.join([re.escape(key) + r'[\s\r\n]*' for key in d])

# Validation function for area based on the regex pattern
def is_valid_area(area):
    if isinstance(area, str):
        return re.match(regex_patterns, area) is not None
    else:
        return False


df_area = df[['area_preprocessed']].drop_duplicates()
df_area['valid_area'] = df_area['area_preprocessed'].apply(is_valid_area)
df_area.info()
df_area.head()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 992
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   area_preprocessed  111 non-null    object
 1   valid_area         112 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 1.9+ KB


,area_preprocessed,valid_area
0,<NA>,False
1,E2,False
2,ภาคตะวันตก,True
3,ภาคเหนือ,True
4,ภาคกลาง,True


In [5]:
df_area['valid_area'].sum()

6

In [6]:
# Q3 Validation functions for phone numbers based on the specified rules

# Rule 1: Phone numbers should contain between 9 and 10 digits, start with "0", and may contain "-" between digits.
def is_valid_phone_rule1(phone):
    if isinstance(phone, str):
        # Removing '-' and checking the length
        phone_digits = phone.replace('-', '')
        return phone.startswith("0") and 9 <= len(phone_digits) <= 10
    else:
        return False

# Rule 2: Phone numbers starting with "02" should have 9 digits, and all others should have 10 digits.
def is_valid_phone_rule2(phone):
    if isinstance(phone, str):
        # Removing '-' for length check
        phone_digits = phone.replace('-', '')
        if phone.startswith("02"):
            return len(phone_digits) == 9
        else:
            return len(phone_digits) == 10
    else:
        return False

df_phone = df[['phone']].drop_duplicates()
df_phone['valid_phone_rule1'] = df_phone['phone'].apply(is_valid_phone_rule1)
df_phone['valid_phone_rule2'] = df_phone['phone'].apply(is_valid_phone_rule2)
df_phone.info()
df_phone.head()
print(df_phone['valid_phone_rule1'].sum(),df_phone['valid_phone_rule2'].sum())

<class 'pandas.core.frame.DataFrame'>
Index: 995 entries, 0 to 999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   phone              994 non-null    string
 1   valid_phone_rule1  995 non-null    bool  
 2   valid_phone_rule2  995 non-null    bool  
dtypes: bool(2), string(1)
memory usage: 17.5 KB
603 321


In [16]:
#Q4 Validation functions for ISO date
def is_valid_date(date):
    try:
        if isinstance(date, str):
            pd.to_datetime(date, format='%Y-%m-%d %H:%M:%S', errors='raise')
            return True
        else:
            return False
    except:
        return False


df_date = df[['date']].drop_duplicates()
# df_date = df[['date']]
df_date['valid_date'] = df_date['date'].apply(is_valid_date)
df_date.info()
df_date.head()

<class 'pandas.core.frame.DataFrame'>
Index: 815 entries, 0 to 998
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        814 non-null    string
 1   valid_date  815 non-null    bool  
dtypes: bool(1), string(1)
memory usage: 13.5 KB


,date,valid_date
0,2018-10-12 00:00:00,True
1,2019-07-05 00:00:00,True
2,2021-01-11 00:00:00,True
3,2019-02-12 00:00:00,True
4,mkc5K1v45,False


In [17]:
df_date['valid_date'].sum()

720